In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, precision_recall_curve
from matplotlib import pyplot as plt
from sklearn.metrics import plot_precision_recall_curve
import numpy as np
from sklearn.model_selection import GridSearchCV
import joblib
from tokenizer import tokenize_sentence
from tokenizer import tokenize_sentence_ru

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
df = pd.read_csv("./data/dataset_test_english.csv", sep=",")

In [ ]:
train_df = pd.read_csv("./data/dataset_train_english.csv", sep=",")

In [7]:
df.dtypes

Unnamed: 0      int64
comment        object
type          float64
dtype: object

In [ ]:
df.head(5)

In [ ]:
df["type"] = df["type"].apply(int)
train_df["type"] = train_df["type"].apply(int)

In [ ]:
df[df['type'] == 1].head(5)

In [ ]:
df[df['type'] == 0].head(5)

In [ ]:
sentence_example = df.iloc[1]["comment"]
tokenize_sentence(sentence_example)

In [ ]:
vectorizer = TfidfVectorizer(tokenizer=tokenize_sentence)

In [ ]:
features = vectorizer.fit_transform(train_df["comment"])

In [ ]:
model = LogisticRegression(random_state=0)
model.fit(features, train_df["type"])

In [ ]:
model.predict(features[0])

In [ ]:
train_df["comment"].iloc[0]

In [ ]:
model_pipeline = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer=tokenize_sentence)),
    ("model", LogisticRegression(random_state=0))
]
)

In [ ]:
model_pipeline.fit(train_df["comment"], train_df["type"])

In [ ]:
pos_text = df['comment'].iloc[0]
neg_text = df[df['type']==0]['comment'].iloc[0]

In [ ]:
pos_text

In [ ]:
neg_text

In [ ]:
model_pipeline.predict([pos_text])

In [ ]:
model_pipeline.predict([neg_text])

In [ ]:
precision_score(y_true=df["type"], y_pred=model_pipeline.predict(df["comment"]))

In [ ]:
recall_score(y_true=df["type"], y_pred=model_pipeline.predict(df["comment"]))

In [ ]:
prec, rec, thresholds = precision_recall_curve(y_true=df["type"], probas_pred=model_pipeline.predict_proba(df["comment"])[:, 1])

In [ ]:
plot_precision_recall_curve(estimator=model_pipeline, X=df["comment"], y=df["type"])

In [ ]:
min_threshold = np.where(prec > 0.95)[0][0]
min_threshold

In [ ]:
np.where(prec > 0.95)

In [ ]:
thresholds[min_threshold]

In [ ]:
precision_score(y_true=df["type"], y_pred=model_pipeline.predict_proba(df["comment"])[:, 1] > thresholds[min_threshold])

In [ ]:
recall_score(y_true=df["type"], y_pred=model_pipeline.predict_proba(df["comment"])[:, 1] > thresholds[min_threshold])

In [ ]:
grid_pipeline = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer=tokenize_sentence)),
    ("model", 
     GridSearchCV(
        LogisticRegression(random_state=0),
        param_grid={'C': [0.1, 1, 10.]},
        cv=3,
         verbose=4
        )
    )
])

In [ ]:
grid_pipeline.fit(train_df["comment"], train_df["type"])

In [ ]:
model_pipeline_c_10 = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer=tokenize_sentence)),
    ("model", LogisticRegression(random_state=0, C=10.))
]
)

In [ ]:
model_pipeline_c_10.fit(train_df["comment"], train_df["type"])

In [ ]:
prec_c_10, rec_c_10, thresholds_c_10 = precision_recall_curve(y_true=df["type"], probas_pred=model_pipeline_c_10.predict_proba(df["comment"])[:, 1])

In [ ]:
min_thresholdC10 = np.where(prec_c_10 > 0.95)[0][0]

In [ ]:
np.where(prec_c_10 > 0.95)

In [ ]:
precision_score(y_true=df["type"], y_pred=model_pipeline_c_10.predict_proba(df["comment"])[:, 1] > thresholds_c_10[min_thresholdC10])

In [ ]:
recall_score(y_true=df["type"], y_pred=model_pipeline_c_10.predict_proba(df["comment"])[:, 1] > thresholds_c_10[min_thresholdC10])

In [ ]:
joblib.dump(model_pipeline, 'modelpipeline.joblib')

In [ ]:
joblib.dump(model_pipeline_c_10, 'modelpipeline_c_10.pkl')

In [2]:
df_ru = pd.read_csv('./data/dataset_train_ru.csv', sep=',')

In [3]:
df_ru['type'] = df_ru['type'].apply(int)

In [4]:
df_ru.dtypes

Unnamed: 0       int64
Unnamed: 0.1     int64
comment         object
type             int64
dtype: object

In [5]:
df_ru.shape

(6639, 4)

In [6]:
df_ru.head(5)

,Unnamed: 0,Unnamed: 0.1,comment,type
0,0,0,Bromwell High - мультяшная комедия.Он бежал в ...,1
1,1,1,"Бездомность (или бездействие, как заявил Джорд...",1
2,2,2,"Блестящий закон, действующий Лесли Энн Уоррен....",1
3,3,3,"Это легко, это самый недооцененный фильм Inn B...",1
4,4,4,Это не типичный фильм Mel Brooks.Это было гора...,1


In [7]:
model_pipeline_ru = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer=tokenize_sentence_ru)),
    ("model", LogisticRegression(random_state=0))
]
)

In [8]:
model_pipeline_ru.fit(df_ru['comment'], df_ru['type'])

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function tokenize_sentence_ru at 0x00000208B2B9AA60>)),
                ('model', LogisticRegression(random_state=0))])

In [9]:
precision_score(y_true=df_ru["type"], y_pred=model_pipeline_ru.predict(df_ru["comment"]))

0.9618416903185115

In [10]:
recall_score(y_true=df_ru["type"], y_pred=model_pipeline_ru.predict(df_ru["comment"]))

0.9537210756722951

In [11]:
joblib.dump(model_pipeline_ru, 'modelpipeline_ru.pkl')

['modelpipeline_ru.pkl']